In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

In [8]:
dataset = pd.read_csv('./data/athlete_events.csv')
dataset.info()
dataset.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     261642 non-null  float64
 4   Height  210945 non-null  float64
 5   Weight  208241 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  int64  
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Medal   39783 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [9]:
print(dataset.describe())
print()
print(dataset.isna().sum())
print()
df = pd.DataFrame(dataset, columns=['Height', 'Team'])
df = dataset[['Height', 'Team']].sort_values(['Height'],ascending=False)


percentOfNaAge = ((dataset['Age'].isna().sum() / 271116) * 100)
percentOfNaHeight = ((dataset['Height'].isna().sum() / 271116) * 100)
percentOfNaWeight = ((dataset['Weight'].isna().sum() / 271116) * 100)
percentOfNaMedals = ((dataset['Medal'].isna().sum() / 271116) * 100)
value = {'a': percentOfNaAge, 'h': percentOfNaHeight, 'w': percentOfNaWeight, 'm': percentOfNaMedals}
print()
print('''
PercentOfNaAge: {a} 

PercentOfNaHeight: {h} 

PercentOfNaWeight: {w}

PercentOfNaMedals: {m}'
'''.format(**value))


                  ID            Age         Height         Weight  \
count  271116.000000  261642.000000  210945.000000  208241.000000   
mean    68248.954396      25.556898     175.338970      70.702393   
std     39022.286345       6.393561      10.518462      14.348020   
min         1.000000      10.000000     127.000000      25.000000   
25%     34643.000000      21.000000     168.000000      60.000000   
50%     68205.000000      24.000000     175.000000      70.000000   
75%    102097.250000      28.000000     183.000000      79.000000   
max    135571.000000      97.000000     226.000000     214.000000   

                Year  
count  271116.000000  
mean     1978.378480  
std        29.877632  
min      1896.000000  
25%      1960.000000  
50%      1988.000000  
75%      2002.000000  
max      2016.000000  

ID             0
Name           0
Sex            0
Age         9474
Height     60171
Weight     62875
Team           0
NOC            0
Games          0
Year           0


In [14]:
df_describe = dataset.groupby('Team').describe()

In [25]:
df_describe['Age'].sort_values('mean', ascending=False)[:50]

,count,mean,std,min,25%,50%,75%,max
Team,,,,,,,,
Pirouette-31,1.0,71.000000,NaN,71.0,71.00,71.0,71.00,71.0
Pirouette-5,2.0,71.000000,0.000000,71.0,71.00,71.0,71.00,71.0
Whisper,1.0,67.000000,NaN,67.0,67.00,67.0,67.00,67.0
Ariette-8,1.0,62.000000,NaN,62.0,62.00,62.0,62.00,62.0
Ariette-10,2.0,62.000000,0.000000,62.0,62.00,62.0,62.00,62.0
Dow Jones,1.0,58.000000,NaN,58.0,58.00,58.0,58.00,58.0
Cincinnati Archers,6.0,57.666667,8.213810,44.0,55.00,58.5,62.00,68.0
Potomac Archers,4.0,57.500000,7.141428,48.0,54.00,59.5,63.00,63.0
Sibindi-2,1.0,55.000000,NaN,55.0,55.00,55.0,55.00,55.0


In [39]:
dataset[['Age','Team']].groupby('Team').Age.apply(lambda x: x.mode())[:50]

Team                           
30. Februar                   0    23.0
                              1    44.0
A North American Team         0    38.0
                              1    42.0
                              2    44.0
Acipactli                     0    44.0
                              1    46.0
                              2    52.0
Acturus                       0    27.0
Afghanistan                   0    20.0
Akatonbo                      0    36.0
                              1    41.0
                              2    44.0
Alain IV                      0    40.0
                              1    50.0
                              2    54.0
Albania                       0    21.0
Alcaid                        0    28.0
                              1    29.0
                              2    33.0
Aldebaran                     0    25.0
                              1    40.0
Aldebaran II                  0    36.0
                              1    39.0
Aletta  

In [50]:
dataset['Age'].groupby(dataset['Age']).count().sort_values(ascending=False)[:60]

Age
23.0    21875
24.0    21720
22.0    20814
25.0    19707
21.0    19164
26.0    17675
27.0    16025
20.0    15258
28.0    14043
19.0    11643
29.0    11463
30.0     9488
18.0     8152
31.0     7559
32.0     6246
17.0     5376
33.0     4800
34.0     3985
16.0     3852
35.0     3133
36.0     2503
15.0     2203
37.0     1953
38.0     1612
39.0     1405
40.0     1210
41.0      953
42.0      866
14.0      837
43.0      793
44.0      683
45.0      584
46.0      429
47.0      408
48.0      407
49.0      362
50.0      278
52.0      244
53.0      200
51.0      199
13.0      187
54.0      162
56.0      131
55.0      104
60.0       88
Name: Age, dtype: int64

In [51]:
dataset['Age'].mode()

0    23.0
dtype: float64